<a href="https://colab.research.google.com/github/louisacheong/nlp/blob/main/genai-qa-lora/genai-qa-lora_demo_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Domain-specific Q&A with Hugging Face and LoRA
This notebook demonstrates a simple pipeline for fine-tuning a transformer-based model (e.g., LLaMA/GPT) on a domain-specific Q&A dataset using LoRA.

In [1]:
# Install required libraries
!pip install -q transformers datasets peft accelerate bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

## Step 1: Load a Pretrained Model and Tokenizer

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "tiiuae/falcon-7b-instruct"  # Or any other model that supports LoRA
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

## Step 2: Apply LoRA with PEFT

In [3]:
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],  # Adjust based on model architecture
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 6,924,080,000 || trainable%: 0.0341


## Step 3: Load or Create a Simple Q&A Dataset

In [4]:
from datasets import Dataset

# Example: A few Q&A pairs for training
data = {
    "question": [
        "What documents are needed for an insurance claim?",
        "How long is the cooling-off period?",
    ],
    "answer": [
        "Typically, a completed claim form, policy number, and supporting invoices or reports.",
        "Usually 14 days from the date of purchase or agreement."
    ]
}

dataset = Dataset.from_dict(data)
dataset = dataset.map(lambda x: {"text": f"Q: {x['question']} A: {x['answer']}"})
dataset

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'text'],
    num_rows: 2
})

## Step 4: Train or Fine-tune the Model (Optional Demo Step)

In [5]:
# Skipping full training for demo – insert Trainer setup here if needed.

## Step 5: Run Inference

In [6]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
pipe("Q: What documents are needed for an insurance claim? A:")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': "Q: What documents are needed for an insurance claim? A: Some documents you'll need for an insurance claim include a proof of loss, bills, and an inventory of damaged or destroyed property. You'll also need to provide an appraisal or estimate of your losses."}]

## Step 6: Deploy with Gradio

In [7]:
import gradio as gr

def qa_bot(prompt):
    result = pipe(f"Q: {prompt} A:")[0]["generated_text"]
    return result.split("A:")[-1].strip()

gr.Interface(fn=qa_bot, inputs="text", outputs="text", title="Domain Q&A Bot").launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33f1e87ce486ef9d78.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
